# Simple OpenPhase example

In [1]:
gInterpreter->AddIncludePath("/home/builder/include");
gInterpreter->Load("/home/builder/lib/libOPhase.so");
//gInterpreter->GetIncludePath();

This is a ROOT notebook following this [example](https://root.cern.ch/notebooks/HowTos/HowTo_ROOT-Notebooks.html).


In [2]:
#include "Settings.h"
#include "BoundaryConditions.h"
#include "InterfaceEnergy.h"
#include "InterfaceMobility.h"
#include "DoubleObstacle.h"
#include "PhaseField.h"
#include "InterfaceField.h"
#include "DrivingForce.h"
#include "InterfaceEnergy.h"
#include "Initializations.h"

### Initialization and reading input parameters

In [3]:
openphase::Settings            OPSettings;

OPSettings.Initialize();
OPSettings.ReadInput();

openphase::InterfaceMobilityIdentical   Mu;
openphase::BoundaryConditions  BC(OPSettings);
openphase::PhaseField          Phi(OPSettings);
openphase::DrivingForce        dG;
openphase::InterfaceField      Psi(OPSettings);
openphase::DoubleObstacle      DO(OPSettings);
openphase::InterfaceEnergyIdentical Sigma;

Sigma.Initialize(OPSettings);
Mu.Initialize(OPSettings);
dG.Initialize(OPSettings);

dG.ReadInput();
BC.ReadInput();

double t=0, dt=OPSettings.dt * 0.01;

openphase::Initializations::Single(Phi, 0, BC, OPSettings);
openphase::Initializations::Sphere(Phi, 1, 8.5, 25,25,25,  BC, OPSettings);


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
  OpenPhase

  An open-source phase-field simulation library
  www.openphase.de

  Core development: 
  Interdisciplinary Centre for Advanced Materials Simulation (ICAMS) 
  Ruhr University Bochum, Germany 
  2009 - 2016

  Licensed under GNU GPLv3
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

Settings                                : Initialized
------------------------------------------------------------
------------------------------------------------------------
Global project parameters source        : ProjectInput/ProjectInput.opi
Simulation Title                        : Multi
--< Model parameters >--------------------------------------
Units of length                         : m
Units of mass                           : kg
Units of time                           : s
Energy units                            : J
Number of OpenMP Threads                : 1
Number of Time Steps                    : 20
Init

### Define a function to calculate statistics
Using the `%%cpp` [magic](https://github.com/root-project/root/blob/master/bindings/pyroot/JupyROOT/kernel/magics/cppmagic.py).


In [4]:
%%cpp -d

double f_v(int phiid=1)
{// calculate volume fraction of phase `phiid`
    double phisum=0.0;
    for (int j=0; j<Phi.Fields.tot_size(); j++)
        phisum+=Phi.Fields[j].get(phiid);
    return phisum/Phi.Fields.tot_size();
};

### Time advancing

In [5]:
for(int i=0; i<50; i++)
{
    Sigma.CalculateCubic(Phi);
    Mu.CalculateCubic(Phi);
    DO.GetPsi(Phi, Sigma, Mu, Psi);
    dG.Average(Phi, BC);
    dG.MergePsi(Phi, Sigma, Mu, Psi);
    Psi.Normalize(Phi, BC, dt);
    Psi.Merge(Phi, BC, dt);
    t+=dt;
}
t

(double) 0.00050000000


### Plot $\phi$ profile and status parameters

The following code is modified from [this example](https://root.cern.ch/root/htmldoc/guides/users-guide/ROOTUsersGuide.html#tgraph).

The line profile of $\phi_1(y)$ is ploted along the line $(25,0,25)$--$(25,50,25)$.

In [6]:
%jsroot
{
    Int_t n = OPSettings.Nx;
    Double_t x[n], y[n];
    for (Int_t i=0;i<n;i++) {
        x[i]= i * OPSettings.dx;
        y[i]= Phi.Fields(25,i,25).get(1);
    }
    
    TGraph *gr  = new TGraph(n,x,y);
    TCanvas *c1 = new TCanvas("c1","Graph Draw Options",
                             200,10,600,300);

    // draw the graph with axis, continuous line, and put a * at each point
    gr->Draw("AC*");
    c1->Draw();
    double fv=f_v();
    cout<< "fv[1]="<< fv << endl 
        << "Interface energy = " << DO.Energy(Phi, Sigma);
}

fv[1]=0.0105743
Interface energy = 25234.3